<a href="https://colab.research.google.com/github/kimhwijin/KFood_Models/blob/master/KFood_Dataset_Unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import os
import zipfile
import numpy as np
import csv


In [34]:
DATASET_PATH = Path(os.getcwd())

In [8]:
dest_path = DATASET_PATH / "kfood"
if not dest_path.exists():
    with zipfile.ZipFile(DATASET_PATH / "kfood.zip") as kfood:
        zipInfo = kfood.infolist()
        for member in zipInfo:
            try:
                print(member.filename.encode('cp437').decode('euc-kr', 'ignore'))
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                kfood.extract(member, dest_path)
            except:
                #print(source_file)
                raise Exception('what?!')


구이.zip
국.zip
기타.zip
김치.zip
나물.zip
떡.zip
만두.zip
면.zip
무침.zip
밥.zip
볶음.zip
쌈.zip
음청류.zip
장.zip
장아찌.zip
적.zip
전.zip
전골.zip
조림.zip
죽.zip
찌개.zip
찜.zip
탕.zip
튀김.zip
한과.zip
해물.zip
회.zip


In [9]:
root_path = DATASET_PATH / "kfood"

zipfile_paths = list(root_path.glob("*.zip"))

for fname in zipfile_paths:
    fname = fname.stem
    dest_path = root_path
    print(fname)
    with zipfile.ZipFile(root_path / "{}.zip".format(fname), "r") as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            member.filename = member.filename.encode('utf8').decode("utf8", "ignore")
            zf.extract(member, dest_path)   


탕
볶음
조림
무침
만두
김치
떡
전골
튀김
구이
음청류
장아찌
장
국
회
찌개
죽
나물
쌈
기타
밥
한과
찜
해물
전
적
면


In [35]:
cat_names = os.listdir(root_path)
cat_names = [root_path / name for name in cat_names if os.path.isdir(os.path.join(root_path, name))]
print(cat_names)

[PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/적'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/회'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/무침'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/만두'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/구이'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/나물'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/탕'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/쌈'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/국'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/장'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/면'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/전골'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/떡'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/전'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/해물'), PosixP

In [36]:
from PIL import Image
path = root_path / "국/계란국/Img_014_0000.jpg"

with Image.open(path) as image:
    np_image = np.array(image)

print(np_image.shape)


(645, 430, 3)


In [37]:
def create_example_protoc(image, label):
    serialized_image = tf.io.serialize_tensor(image)
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                "image" : tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_image.numpy()]))
            }
        )
    )

In [38]:
label_paths = list(root_path.glob("*/*"))
label_name_index_dict = {}
for index, label_path in enumerate(label_paths):
    label_name_index_dict[label_path.stem] = index
print(label_name_index_dict.items())

with open('class_label.csv','w') as f:
    w = csv.writer(f)
    w.writerow(label_name_index_dict.keys())
    w.writerow(label_name_index_dict.values())

dict_items([('떡꼬치', 236), ('편육', 200), ('후라이드치킨', 207), ('콩자반', 183), ('젓갈', 217), ('양념치킨', 250), ('과메기', 226), ('피자', 203), ('육회', 204), ('물회', 274), ('알밥', 186), ('누룽지', 245), ('김치볶음밥', 172), ('김밥', 164), ('비빔밥', 265), ('새우볶음밥', 233), ('유부초밥', 224), ('잡곡밥', 208), ('주먹밥', 270), ('회무침', 199), ('꽈리고추무침', 192), ('도라지무침', 194), ('콩나물무침', 279), ('홍어무침', 162), ('잡채', 146), ('도토리묵', 231), ('만두', 188), ('더덕구이', 190), ('떡갈비', 181), ('갈비구이', 173), ('훈제오리', 286), ('장어구이', 258), ('삼겹살', 144), ('불고기', 284), ('황태구이', 262), ('조기구이', 234), ('조개구이', 147), ('갈치구이', 263), ('고등어구이', 289), ('곱창구이', 216), ('닭갈비', 267), ('미역줄기볶음', 175), ('고사리나물', 158), ('가지볶음', 257), ('애호박볶음', 287), ('숙주나물', 214), ('시금치나물', 169), ('갈비탕', 220), ('매운탕', 223), ('감자탕', 151), ('추어탕', 196), ('곰탕_설렁탕', 232), ('삼계탕', 160), ('보쌈', 278), ('육개장', 290), ('북엇국', 215), ('콩나물국', 244), ('무국', 179), ('계란국', 157), ('미역국', 269), ('떡국_만두국', 251), ('시래기국', 165), ('간장게장', 249), ('양념게장', 225), ('물냉면', 154), ('수제비', 156), ('콩국수', 195), ('막국수', 170

In [19]:
from contextlib import ExitStack

tfrecords_path = root_path / "tfrecords"
if not tfrecords_path.exists():
    os.mkdir(tfrecords_path)


def write_tfrecords(filepath, n_shards=1): #파일 조각 개수

    print(filepath.stem, sep=' ')
    paths = [ str(tfrecords_path) + "/{}.tfrecord-{:05d}-{:05d}".format(filepath.stem, index+1, n_shards) for index in range(n_shards)]
    image_paths = get_image_paths(filepath)
    with ExitStack() as stack:
        writers = [stack.enter_context(tf.io.TFRecordWriter(path)) for path in paths]
        for index, image_path in enumerate(image_paths):
            shard = index % n_shards
            with Image.open(path) as image:
                np_image = np.array(image)
                example = create_example_protoc(np_image, label_name_index_dict[filepath.stem])
                writers[shard].write(example.SerializeToString())
    return paths

def get_image_paths(filepath, image_formats=("*.j*", "*.J*", "*.png", "*.PNG", "*.b*", "*.B*", '*.g*', '*.G*')):
    paths = []
    for image_format in image_formats:
        paths += list(filepath.glob(image_format))
    return sorted(paths)

In [20]:
for label_path in label_paths:
    print(os.path.getsize(write_tfrecords(label_path)[0]) // 1000000)
    

떡꼬치
911
편육
832
후라이드치킨
832
콩자반
832
젓갈
832
양념치킨
832
과메기
834
피자
832
육회
832
물회
842
알밥
832
누룽지
832
김치볶음밥
832
김밥
832
비빔밥
832
새우볶음밥
832
유부초밥
832
잡곡밥
832
주먹밥
832
회무침
835
꽈리고추무침
832
도라지무침
832
콩나물무침
832
홍어무침
832
잡채
832
도토리묵
834
만두
832
더덕구이
832
떡갈비
832
갈비구이
832
훈제오리
832
장어구이
832
삼겹살
832
불고기
832
황태구이
832
조기구이
832
조개구이
832
갈치구이
832
고등어구이
832
곱창구이
832
닭갈비
832
미역줄기볶음
832
고사리나물
832
가지볶음
832
애호박볶음
832
숙주나물
832
시금치나물
832
갈비탕
832
매운탕
833
감자탕
834
추어탕
834
곰탕_설렁탕
833
삼계탕
834
보쌈
834
육개장
832
북엇국
832
콩나물국
832
무국
832
계란국
832
미역국
832
떡국_만두국
832
시래기국
832
간장게장
832
양념게장
832
물냉면
832
수제비
832
콩국수
832
막국수
832
열무국수
832
비빔냉면
834
짜장면
832
쫄면
832
칼국수
832
잔치국수
832
라면
832
짬뽕
832
곱창전골
832
꿀떡
834
경단
832
송편
832
파전
832
계란말이
833
동그랑땡
832
호박전
832
생선전
832
김치전
832
계란후라이
833
감자전
832
멍게
906
산낙지
886
순대
834
찜닭
834
계란찜
834
닭볶음탕
833
꼬막찜
835
김치찜
832
해물찜
832
갈비찜
832
족발
832
수육
835
깻잎장아찌
833
한과
840
약과
834
약식
836
식혜
833
수정과
832
라볶이
832
고추장진미채볶음
832
멸치볶음
832
제육볶음
832
소세지볶음
834
어묵볶음
832
건새우볶음
832
오징어채볶음
901
두부김치
832
떡볶이
832
주꾸미볶음
833
감자채볶음
832
코다

[PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/적/떡꼬치'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타/편육'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타/후라이드치킨'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타/콩자반'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타/젓갈'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타/양념치킨'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타/과메기'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타/피자'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/회/육회'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/회/물회'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥/알밥'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥/누룽지'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥/김치볶음밥'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥/김밥'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥/비빔밥'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥/새우볶음밥'),